# Ayudantía 4 - Sistemas Recomendadores: Content based

**Nombre(s):**

## Setup

**Paso 1:** Descarga de archivos que serán utilizados posteriormente.

*   Recursos:
  * `dictionary.p`
  * `dictionary-stemm.p`
  * `tfidf_model.p`
  * `tfidf_model-stemm.p`
*   Dataset:
  *  `corpus1.csv`

In [0]:
# Descarga de recursos
!curl -L -o 'resources.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%204/files/resources.tar.gz?raw=true'

# Descompresión del archivo
!tar -xvf resources.tar.gz

In [0]:
# Descarga del dataset
!curl -L -o 'dataset.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%204/files/dataset.tar.gz?raw=true'

# Descompresión del archivo
!tar -xvf dataset.tar.gz

**Paso 2:** Para este práctico es necesario instalar las siguentes dependencias:

In [0]:
!pip install nltk
!pip install sklearn
!pip install gensim
!pip install pandas
!pip install numpy

In [0]:
import string

import gensim
import nltk
import numpy as np
import pandas as pd
import sklearn

from collections import Counter
from os.path import isfile
from textwrap import wrap

from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

## Preprocesamiento de datos

Lo primero es descargar las librerías de NLTK necesarias:

In [0]:
nltk.download('punkt')

Para comenzar cargaremos el set de datos en un *dataframe* de Pandas, e imprimimos los 5 primeros registros para visualizar la estructura de los datos.

In [0]:
corpus_df = pd.read_csv('./corpus1.csv', sep='\t',
                        header=None, encoding='latin')
corpus_df.columns = ['id', 'title', 'abstract']
corpus_df = corpus_df[['id', 'title', 'abstract']]
corpus_df.head(5)

Lo siguiente es implementar una función que transforme texto no estructurado a una lista de *tokens* procesados.

In [0]:
stemm = False
stemmer = PorterStemmer()


def get_tokens(text):
    lowers = text.lower()
    no_punctuation = lowers.translate(
        {ord(c): None for c in string.punctuation})
    tokens = nltk.word_tokenize(no_punctuation)
    if stemm:
        tokens = map(stemmer.stem, tokens)

    return tokens


get_tokens("I'm a super student for recommender systems!")

Ahora se tiene que generar un diccionario con todas las palabras del *corpus*. Se recomienda revisar la documentación de gensim y leer cómo usar los diccionarios: [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html)

In [0]:
dict_file = './resources/dictionary-stemm.p' if stemm else './resources/dictionary.p'
if isfile(dict_file):
    dictionary = corpora.dictionary.Dictionary().load(dict_file)
else:
    dictionary = corpora.dictionary.Dictionary(documents=corpus_df.tokenised_abstract.tolist())
    dictionary.save(dict_file)
    
corpus_df['tokenized_abstract'] = corpus_df.abstract.map(get_tokens)
corpus_df.head(5)

In [0]:
corpus_df['bow'] = corpus_df.tokenized_abstract.map(dictionary.doc2bow)
#del corpus_df['tokenized_abstract']
corpus = corpus_df['bow'].tolist()
corpus_df.head(5)

## Tf-idf

Esto fue trabjado y comentado durante el último práctico:

In [0]:
tfidf_model_file = 'resources/tfidf_model-stemm.p' if stemm else 'resources/tfidf_model.p'
if isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)
else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

corpus_df['tf_idf'] = tfidf_model[corpus_df.bow.tolist()]
corpus_df.head(5)

## LDA

A continuación utilizaremos el modelo LDA para identificar 10 tópicos sobre los documentos del dataset:

In [0]:
topic_number = 10

lda_model = models.LdaModel(corpus, num_topics=topic_number,
                            id2word=dictionary, passes=5, iterations=200)
corpus_df['lda'] = lda_model[corpus_df.bow.tolist()]
corpus_df.head(5)

**Pregunta:** Explique qué representa la columna `lda`, ¿qué significan cada tupla de números?

**Respuesta:**

En la siguiente celda se mostrarán 10 tópicos del modelo LDA.

In [0]:
lda_model.print_topics(10)

**Pregunta:** ¿Qué representa lo impreso en la celda anterior?

**Respuesta:**

**Pregunta:** A su parecer, ¿son buenos los tópicos encontrados por el modelo? ¿cómo se podrían mejorar?

**Respuesta:**

## Generar recomendaciones

En esta sección se implementan las funciones necesarias para poder generar recomendaciones dado lo que un usuario ha consumido. De manera artificial, se "samplearán" 3 documentos aleatorios que representarán al usuario objetivo (`sample`). Luego tendrás que generar diferentes recomendaciones y evaluar los resultados.

In [0]:
# Random users
samples = corpus_df.sample(3)
samples_ids = []

for n, (ix, paper) in enumerate(samples.iterrows()):
    samples_ids.append(ix)
    idx, title, abstract, bow, tf_idf, lda = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf', 'lda']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

In [0]:
# Recommendation functions

N = len(dictionary)


def to_sparse(matrix):
    return csr_matrix([
        gensim.matutils.sparse2full(row, length=N)
        for row in matrix
    ])


def make_recommendations(model, metric, neighbors):
    M = len(corpus)

    X = to_sparse(corpus_df[model].tolist())
    document_index = NearestNeighbors(
        n_neighbors=(neighbors + 1),
        algorithm='brute',
        metric=metric).fit(X)
    return document_index


def print_recommendations(indexes, model):
    for n, (ix, paper) in enumerate(samples.iterrows()):
        dists, neighbors = indexes.kneighbors([gensim.matutils.sparse2full(paper[model], length=N)])
        print(paper['title'])
        print('')
        print('Documentos cercanos: ')
        i = 1
        for neighbour in neighbors[0]:
            if ix != neighbour:
                line = str(i) + ". " + corpus_df.iloc[neighbour]['title']
                print(line)
                i += 1
        print('\n')

A continuación deberá utilizar las funciones implementadas anteriormente para generar nuevas recomendaciones variando los parámetros del modelo. **Agregue nuevas celdas para cada implementación y/o pregunta.**


Aquí hay 2 ejemplos, puede crear más celdas para hacer las pruebas necesarias:

In [0]:
# Recommendation example: TF-IDF
doc_idx = make_recommendations('tf_idf', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf')

In [0]:
# Recommendation example: LDA
doc_idx = make_recommendations('lda', 'euclidean', 5)
print_recommendations(doc_idx, 'lda')

**Pregunta:** Ejecute el modelo utilizando como representación tf-idf y una métrica de distancia euclideana. Modifique el parámetro nearest_neighbors a [5, 10, 20]. ¿Qué efecto tiene este cambio en el modelo en las recomendaciones observadas?

**Respuesta:**

**Pregunta:** Eligiendo un valor fijo para *nearest neighbors* y utilizando representación tf-idf, ejecute el modelo con métrica de distancia *cosine*.¿Qué efecto tiene la métrica de distancia en las recomendaciones observadas?

**Respuesta:**


**Pregunta:** Eligiendo un valor fijo de nearest_neighbors y modelo lda ¿Qué efecto tiene el usar LDA versus TF-IDF en las recomendaciones observadas bajo la misma métrica de distancia?

**Respuesta:**

**Pregunta:** Pruebe nuevamente con LDA usando 5 tópicos y con 20 tópicos ¿qué efecto tiene el número de tópicos en las recomendaciones observadas?

**Respuesta:**

## Stop words

A continuación, intentaremos mejorar los resultados obtenidos con LDA eliminando las *stopwords*. ¿Qué son las *stopwords*? Son palabras vacías, sin significado, que no aportan (de manera significativa) al sentido de una frase, como los artículos, pronombres, etc.

In [0]:
nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    filtered_words = [
        word for word in text if word not in stopwords.words('english')
    ]
    return filtered_words

Ahora, repetimos el proceso realizado anteriormente (sin tanto detalle, ya que estos se comentaron previamente)

In [0]:
%%time
# Puede que se demore un poco esta celda
corpus_df['tokenized_abstract_without_stopwords'] = corpus_df.tokenized_abstract.map(remove_stopwords)

In [0]:
corpus_df['bow_without_stopwords'] = corpus_df.tokenized_abstract_without_stopwords.map(dictionary.doc2bow)
del corpus_df['tokenized_abstract_without_stopwords']
corpus = corpus_df['bow_without_stopwords'].tolist()

In [0]:
tfidf_model_file_without_stopwords = 'resources/tfidf_model-stemm.p' if stemm else 'resources/tfidf_model.p'
if isfile(tfidf_model_file):
    tfidf_model_without_stopwords = models.tfidfmodel.TfidfModel().load(tfidf_model_file)
else:
    tfidf_model_without_stopwords = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model_without_stopwords.save(tfidf_model_file_without_stopwords)

corpus_df['tf_idf_without_stopwords'] = tfidf_model_without_stopwords[corpus_df.bow_without_stopwords.tolist()]

In [0]:
topic_number = 10

lda_model_without_stopwords = models.LdaModel(corpus, num_topics=topic_number, id2word=dictionary, passes=5, iterations=200)
corpus_df['lda_without_stopwords'] = lda_model_without_stopwords[corpus_df.bow_without_stopwords.tolist()]

In [0]:
lda_model_without_stopwords.print_topics(10)

**Pregunta:** ¿Qué puede decir de estos nuevos tópicos comparándolos con los obtenidos previamente (sección LDA)?

**Respuesta:**

In [0]:
# Rellocate user

samples = corpus_df.iloc[samples_ids]

for n, (ix, paper) in enumerate(samples.iterrows()):
    idx, title, abstract, bow, tf_idf, lda, bow_without_stopwords, tf_idf_without_stopwords, lda_without_stopwords = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf', 'lda', 'bow_without_stopwords', 'tf_idf_without_stopwords', 'lda_without_stopwords']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

**Pregunta:** Compare las recomendaciones hechas por los métodos cuando quitamos las *stopwords* del diccionario con su versión de las secciones anteriores.

**Respuesta:**

**Pregunta:** ¿Cómo cambian las recomendaciones entre ambos métodos ahora que no consideramos las *stopwords*?

**Respuesta:**

In [0]:
# Recommendation example: TF-IDF without stopwords

doc_idx = make_recommendations('tf_idf_without_stopwords', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf_without_stopwords')

In [0]:
# Recommendation example: LDA without stopwords

doc_idx = make_recommendations('lda_without_stopwords', 'euclidean', 5)
print_recommendations(doc_idx, 'lda_without_stopwords')

**Pregunta:** Realice el siguiente gráfico. Pruebe graficando los items con respecto al tópico con mayor probabilidad de pertenencia, para poder hacer el gráfico deberá usar algún método de reducción de dimensionalidad como PCA o T-SNE a los valores de LDA que están en el dataframe.

Ejemplo:

![Expected plot](https://raw.githubusercontent.com/PUC-RecSys-Class/Syllabus/master/Practico%204/files/plot.png)

In [0]:
# Codigo para generar el grafico

## Otro método: Non-negative Matrix Factorization (NMF)

A continuación, utilizaremos el modelo NMF para generar recomendaciones:

In [0]:
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Tópico #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        print()
    print()


data_samples = corpus_df['abstract'].values

# Formato TF-IDF de sklearn
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data_samples)

# Fit NMF
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5,
          ).fit(tfidf)
nmf_transform_1 = nmf.transform(tfidf)
# Display NMF
print("Tópicos:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
print()
print()

# Fit NMF with KL-Divergence
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5,
          ).fit(tfidf)
# Display NMF with KL-Divergence
print("Tópicos con divergencia KL:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
print()
print()

**Pregunta:** ¿Le parece significativa la diferencia en la calidad de los tópicos al usar la divergencia KL? ¿Son mejores estos tópicos que los obtenidos anteriormente?

**Respuesta:**

Agregamos los valores obtenidos como una columna en el *dataframe*:

In [0]:
corpus_df['NMF'] = [[(i, prob) for i, prob in enumerate(l)] for l in nmf_transform_1.tolist()]

**Pregunta:** Explique qué representa la columna `NMF`, ¿qué significan los valores?

**Respuesta:**

Repetimos el proceso de *sampling* para inspeccionar los resultados:

In [0]:
# Rellocate user

samples = corpus_df.iloc[samples_ids]

for n, (ix, paper) in enumerate(samples.iterrows()):
    idx, title, abstract, bow, tf_idf, lda, bow_without_stopwords, tf_idf_without_stopwords, lda_without_stopwords, nmf = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf', 'lda', 'bow_without_stopwords', 'tf_idf_without_stopwords', 'lda_without_stopwords', 'NMF']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

**Pregunta:** Compare y comente sobre las recomendaciones hechas por los métodos anteriores con las obtenidas usando NMF.

**Respuesta:**

In [0]:
doc_idx = make_recommendations('NMF', 'euclidean', 5)
print_recommendations(doc_idx, 'NMF')